```{contents}
```
## OpenAI Tools Agent

An **OpenAI Tools Agent** is a LangChain agent that uses **OpenAI chat models’ native tool-calling (function calling)** capability to **decide when and how to invoke tools** during a conversation.

> It lets the model produce **structured tool calls** (JSON) instead of free-text guesses, then uses the tool results to generate the final answer.

This agent is purpose-built for **deterministic, production-grade tool use** with OpenAI-compatible chat models from OpenAI.

---

### Why OpenAI Tools Agent Exists

* Native tool calling is **more reliable** than prompt-only tool instructions
* Tools have **strict schemas**, reducing malformed calls
* Better **safety and predictability** than generic agents
* Ideal for **single- or few-step tool usage** (APIs, DB reads, calculations)

---

### Core Capabilities

* Native **JSON tool calls** (no regex parsing)
* Automatic **tool selection**
* Deterministic **argument validation**
* Clean separation of **reasoning → action → result**

---

### Conceptual Flow

```
User Input
   ↓
OpenAI Chat Model (with tools bound)
   ↓
Tool Call (JSON schema)
   ↓
Tool Execution
   ↓
Tool Result
   ↓
Final Answer
```

---

### OpenAI Tools Agent vs ReAct Agent

| Aspect            | OpenAI Tools Agent | ReAct Agent                    |
| ----------------- | ------------------ | ------------------------------ |
| Reasoning loop    | Implicit           | Explicit (Thought/Act/Observe) |
| Tool calls        | Native JSON        | Prompt-driven                  |
| Steps             | Few                | Many                           |
| Predictability    | High               | Medium                         |
| Production safety | High               | Medium                         |

**Rule:** If you need **reliable tool calls**, prefer OpenAI Tools Agent.

---

### When to Use OpenAI Tools Agent

* Calling APIs (read-heavy)
* Fetching live data
* Deterministic computations
* Enterprise automation with guardrails

---

### When NOT to Use It

* Complex multi-hop reasoning
* Conditional branching workflows
* Long-running agent loops
  Use **LangGraph** or **ReAct** instead.

---

### Minimal Demonstration (End-to-End)

#### 1) Define a Tool



In [1]:
from langchain.tools import tool

@tool
def ticket_count(source: str) -> int:
    """Return total number of tickets for a given source."""
    return 128




The decorator:

* Infers name and description
* Builds a strict JSON schema for inputs

---

#### 2) Create a Chat Model



In [2]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)




Low temperature improves tool-call stability.

---

#### 3) Create the Prompt



In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Use tools when necessary to answer accurately."),
    ("human", "{input} {agent_scratchpad}"),
])




No manual tool syntax needed.

---

#### 4) Create the OpenAI Tools Agent



In [6]:
from langchain_classic.agents import create_openai_tools_agent

agent = create_openai_tools_agent(
    llm=chat,
    tools=[ticket_count],
    prompt=prompt
)




LangChain:

* Binds tools to the model
* Enables native tool calling

---

#### 5) Execute with AgentExecutor



In [9]:
from langchain_classic.agents import AgentExecutor

executor = AgentExecutor(
    agent=agent,
    tools=[ticket_count],
    verbose=True
)

result = executor.invoke(
    {"input": "How many tickets are there in Jira?"}
)

print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `ticket_count` with `{'source': 'Jira'}`


128There are 128 tickets in Jira.

> Finished chain.
There are 128 tickets in Jira.




---

### What Happens Internally

1. Model decides a tool is needed
2. Emits a **structured tool call**:

   ```json
   {
     "name": "ticket_count",
     "arguments": { "source": "jira" }
   }
   ```
3. LangChain executes the tool
4. Tool result is fed back
5. Model produces the final answer

---

### Why This Is Safer Than Prompt-Based Tools

* No hallucinated arguments
* No malformed JSON
* Tool calls are **validated**
* Clear audit trail of actions

---

### Guardrails & Best Practices

* **Allowlist tools** (only expose what’s safe)
* **Separate read vs write tools**
* **Low temperature**
* **Max iterations** (prevent loops)
* **Log tool calls**
* **Human approval** for destructive actions

---

### Common Failure Modes (and Fixes)

* **Wrong tool chosen** → improve tool descriptions
* **Too many tool calls** → cap iterations
* **Overuse of tools** → refine system prompt

---

### OpenAI Tools Agent vs MCP

| Aspect     | OpenAI Tools Agent | MCP           |
| ---------- | ------------------ | ------------- |
| Scope      | In-app             | Cross-app     |
| Transport  | In-process         | Client–server |
| Isolation  | Low                | High          |
| Enterprise | Medium             | High          |

Use MCP when tools must be **remote, shared, or sandboxed**.

---

### Interview-Ready Summary

> “An OpenAI Tools Agent uses OpenAI chat models’ native tool-calling to invoke external functions via strict schemas. It provides reliable, deterministic tool usage and is well-suited for production integrations.”

---

### Rule of Thumb

* **Single/few deterministic tool calls → OpenAI Tools Agent**
* **Multi-step reasoning → ReAct**
* **Complex workflows → LangGraph**
* **Cross-service tools → MCP**
